In [1]:
import findspark
findspark.init()


In [2]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-10_2.12:3.1.1,org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.1 pyspark-shell'


In [3]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SparkSession

spark = SparkSession.builder \
                    .master('local[1]') \
                    .appName('tweets') \
                    .getOrCreate()
ssc = StreamingContext(spark.sparkContext, 1)


:: loading settings :: url = jar:file:/usr/local/Cellar/apache-spark/3.1.1/libexec/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/yhua/.ivy2/cache
The jars for the packages stored in: /Users/yhua/.ivy2/jars
org.apache.spark#spark-streaming-kafka-0-10_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a2905c7d-96f8-4f9a-a21a-da23fc54c751;1.0
	confs: [default]
	found org.apache.spark#spark-streaming-kafka-0-10_2.12;3.1.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.1.1 in central
	found org.apache.kafka#kafka-clients;2.6.0 in central
	found com.github.luben#zstd-jni;1.4.8-1 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.2 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.1 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
downloading https://repo1.maven.org/maven2/org/apa

In [4]:
spark

In [6]:
df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "localhost:9092") \
  .option("subscribe", "test1") \
  .load()

query = df.writeStream \
  .outputMode("append") \
  .format("console")\
  .start()
query.awaitTermination()

22/09/06 21:46:36 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/xz/v6q9ksld1134sh8phtl83fqc0000gn/T/temporary-87885c6e-5aba-4ac0-84d9-fc8af54b2792. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


-------------------------------------------
Batch: 0
-------------------------------------------
+---+-----+-----+---------+------+---------+-------------+
|key|value|topic|partition|offset|timestamp|timestampType|
+---+-----+-----+---------+------+---------+-------------+
+---+-----+-----+---------+------+---------+-------------+



-------------------------------------------
Batch: 1
-------------------------------------------
+----+-------------+-----+---------+------+--------------------+-------------+
| key|        value|topic|partition|offset|           timestamp|timestampType|
+----+-------------+-----+---------+------+--------------------+-------------+
|null|[64 64 64 64]|test1|        0|     2|2022-09-06 21:46:...|            0|
+----+-------------+-----+---------+------+--------------------+-------------+

-------------------------------------------
Batch: 2
-------------------------------------------
+----+-----+-----+---------+------+--------------------+-------------+
| key|value|topic|partition|offset|           timestamp|timestampType|
+----+-----+-----+---------+------+--------------------+-------------+
|null| [61]|test1|        0|     3|2022-09-06 21:47:...|            0|
+----+-----+-----+---------+------+--------------------+-------------+



KeyboardInterrupt: 

22/09/06 22:36:18 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1807609 ms exceeds timeout 120000 ms
22/09/06 22:36:18 WARN SparkContext: Killing executors is not supported by current scheduler.
22/09/07 07:13:39 WARN KafkaOffsetReaderConsumer: Error in attempt 1 getting Kafka offsets: 
org.apache.kafka.common.errors.TimeoutException: Timeout of 60000ms expired before the position for partition test1-0 could be determined
22/09/08 04:00:53 WARN KafkaOffsetReaderConsumer: Error in attempt 1 getting Kafka offsets: 
org.apache.kafka.common.errors.TimeoutException: Timeout of 60000ms expired before the position for partition test1-0 could be determined
22/09/08 22:25:25 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-2b7a0721-a71b-45bf-a3f5-d9fad45d0109--1803746445-driver-2-4, groupId=spark-kafka-source-2b7a0721-a71b-45bf-a3f5-d9fad45d0109--1803746445-driver-2] Error connecting to node macbook-pro-2:9092 (id: 2147483647 rack: null)
java.net.

In [ ]:
ssc.textFileStream("./tweets_json/")

In [ ]:
strArray = "Hello World Hello Hadoop Hello spark kafka hive zookeeper hbase flume sqoop scala".split()
strArrayZipped = list(enumerate(strArray))
# print(list(map(lambda x: f"{x[0]} {x[1]}", strArrayZipped)))
strArrayTimestamped = list(map(lambda x: f"{x[0]} {x[1]}", strArrayZipped))
rdd = ssc.sparkContext.parallelize(strArrayTimestamped, len(strArrayTimestamped))
rdd.glom().collect()

In [ ]:
dStream = ssc.queueStream(rdds=[rdd])
counts = dStream.flatMap(lambda line: line.split(" "))\
                     .map(lambda word: (word, 1))\
                     .reduceByKey(lambda a, b: a+b)
counts.pprint(50)
ssc.start()
ssc.awaitTermination()

In [ ]:
sc.stop()
ssc.stop()



In [ ]:
val strArray = "Hello World Hello Hadoop Hello spark kafka hive zookeeper hbase flume sqoop scala"
      .split("\\s+")
      .zipWithIndex
      .map{case (word,timestamp) => s"$timestamp $word"}
val rdd: RDD[String] = ssc.sparkContext.makeRDD(strArray)

    val wordCountStream: ConstantInputDStream[String] = new ConstantInputDStream(ssc,rdd)

    // 流式数据的处理
    wordCountStream.transform{
      rdd=>
      rdd.map{line => (line.split("\\s+")(1).toLowerCase,line)}  //考虑等值连接时的大小写问题
          .leftOuterJoin(blackListRDD)//不用等值连接，因为数据中不一定有黑名单中的词，用左外连接，因为把数据的单词都列出来
          .filter{case (_,(_,rightValue)) => !rightValue.getOrElse(false)}//左外连接的结果里，如果没有连接上就显示None, getOrElse(false)： 取出来就是true，取不处来的就是None，给它一个值false
          .map{case (_,(leftValue,_)) => leftValue}//把过滤的结果进行筛选，只需要（时间戳，单词）这样的数据形式
    }.print(20)